# Use non-linear minimization

Use the Google OR optimization on a smaller number of cases selected at random (not actually in this example, but it doesn't matter as the cases are random). Then use non-linear optimization on the larger file.

Now that I have translated IBM's problem to the Google OR framework _and_ simplified the optimization by making it more realistic, find out when enough is enoughs.

This example uses a file of size 10,000 compared with the original example's 27. Where appropriate, I will scale up with a factor.

| Run | Size | Time (s)  | Value  |
|-----|------|-----------|--------|
|   1 |  27  |  0.036    |    912 |
|   2 |  50  |  0.15     |  1 927 |
|   3 |  75  |  0.25     |  1 975 | 
|   4 |  85  |  0.55     |  2 227 |
|   5 |  90  |   --      |  --    |

What you see above is failure to complete at size 90.

In [1]:
from ortools.linear_solver import pywraplp
import time
from __future__ import print_function
import numpy as np

multiplier = 2
n_obs_new = 100*multiplier
channel_min = 10*multiplier
product_min = 16*multiplier


We have four product types:

  * car loan
  * savings
  * mortgage
  * pension
  
Each product has a different `productValue`: the revenue that can be obtained for the product on average. To get a fair representation of marketing across the various offers, each is allocated a `budgetShare`. 

In [2]:
products = ['Car loan', 'Savings', 'Mortgage', 'Pension']
productValue = [100, 200, 300, 400]
# budgetShare = [0.6, 0.1, 0.2, 0.1]

  
Each product these can be offered over one of the following channels:

  * gift
  * newsletter
  * seminar
  
Each of these channels has different costs, and each has a different _influence factor_. We use the influence to weight the estimated value of the response accordingly.

In [3]:
channels = ['gift', 'newsletter', 'seminar']
cost = [20, 15, 23]
factor = [0.2, 0.05, 0.3]

Budget needs to be less than the available marketing budget of $ \$500$.

In [4]:
availableBudget = 400

Read in the offers data, originally from IBM and massaged. It gives the probability of taking an offer by each customer.

Rather than using the full 10,000, test that it works on a smaller size.

In [5]:
import pandas

product_probs_orig = pandas.read_csv('offers_ibm_pivot.csv')
n_obs_original = product_probs_orig.shape[0]

product_probs = pandas.read_csv('sample_data_10000.csv')
# product_probs = product_probs[product_probs.index > product_probs.shape[0] - n_obs_new]
product_probs = product_probs[product_probs.index < n_obs_new]
n_obs = product_probs.shape[0]

adjustment_factor = n_obs/n_obs_original
availableBudget = availableBudget*adjustment_factor

product_probs.rename(columns={'Unnamed: 0': 'customerid'}, inplace=True)
product_probs.head()

,customerid,name,Car loan,Savings,Mortgage,Pension
0,0,Matthew Harvey,0.000000,0.000000,0.000000,0.000000
1,1,Joshua Wilcox,0.000000,0.000000,0.179932,0.000000
2,2,Yolanda Vasquez,0.330731,0.580556,0.000000,0.000000
3,3,Jessica Alvarado,0.000000,0.630242,0.509746,0.000000
4,4,Gregory Martinez,0.000000,0.320511,0.000000,0.288832


Instantiate the solver as an MIP problem.

In [6]:
solver = pywraplp.Solver('SolveCampaignProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Define the number of customers, the number of offers and the number of channel as $x_{ijk}$.

In [7]:
num_customers = product_probs.shape[0]
num_products = len(products)
num_channels = len(channels)

x = {}

for i in range(num_customers):
    for j in range(num_products):
        for k in range(num_channels):
            x[i, j, k] = solver.IntVar(0, 1, 'x[%i,%i,%i]' % (i, j, k))

### Set up the constraints

  1. Offer only one product per customer.
  2. Do not exceed the budget.
  3. Balance the offers/customers among products.
  

In [8]:
    ## offer only one product per customer
    for i in range(num_customers):
        solver.Add(solver.Sum([x[i, j, k] 
                               for j in range(num_products)
                               for k in range(num_channels)
                              ]) <= 1) 

    ## Do not exceed the budget
    solver.Add(solver.Sum([x[i, j, k]*cost[k]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)
                          ]) <= availableBudget)
    
    ## Balance the offers/customers among products
 #   for j in range(num_products):
 #       solver.Add(solver.Sum([x[i, j, k]
 #                              for i in range(num_customers)
 #                              for k in range(num_channels)
 #           ]) <= budgetShare[j]*solver.Sum([x[i, j, k]
 #                                           for i in range(num_customers)
 #                                           for j in range(num_products)
 #                                           for k in range(num_channels)
 #                                           ]) )
 
# minimums for channel
#    for k in range(num_channels):
#        solver.Add(solver.Sum([x[i, j, k]
#                               for i in range(num_customers)
#                               for j in range(num_products)
#        ]) >= channel_min)
        
#    for j in range(num_products):
#        solver.Add(solver.Sum([x[i, j, k]
#                               for i in range(num_customers)
#                               for k in range(num_channels)
#        ]) >= product_min)


<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x110c1a390> >

### Express the objective

We want to maximize revenue $R$. Here $x_{ijk}$ denotes whether customer $i$ receives an offer for product $j$ over channel $k$, $f_k$ denotes the channel adjustment factor, $v_j$ the product value and $p_{ij}$ the probability that customer $i$ takes up product $j$.

$ \max R = \sum_{ijk} x_{ijk} \times f_k \times v_j \times p_{ij}$

In [9]:
#    solver.Minimize(solver.Sum([cost[i][j] * x[i, j] for i in range(num_workers)
#                                                     for j in range(num_tasks)]))

solver.Maximize(solver.Sum([x[i, j, k]*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]
                           for i in range(num_customers)
                           for j in range(num_products)
                           for k in range(num_channels)]))

### Invoke the solver

In [10]:
# Invoke the solver
t = time.process_time()
sol = solver.Solve()
elapsed_time = time.process_time() - t

Print out the solution. We can print out more information about the constraints.

In [11]:
report = [(channels[k], products[j], product_probs.loc[i, 'name'], x[i, j, k].solution_value()*cost[k],
          x[i, j, k].solution_value()*factor[k]*productValue[j]*product_probs[products[j]].iloc[i]) 
          for i in range(num_customers) 
          for j in range(num_products) 
          for k in range(num_channels)  if x[i, j, k].solution_value() > 0]

report_bd = pandas.DataFrame(report, columns=['channel', 'product', 'customer', 'cost', 'revenue'])

print('Total revenue = %d' % (solver.Objective().Value()))
print('Total budget  = %d' % (report_bd['cost'].sum()) )
print('Time = ', elapsed_time, " seconds.")
display(report_bd)

Total revenue = 7795
Total budget  = 2961
Time =  0.246324  seconds.


,channel,product,customer,cost,revenue
0,seminar,Savings,Yolanda Vasquez,23.0,34.833389
1,seminar,Mortgage,Jessica Alvarado,23.0,45.877099
2,seminar,Pension,Gregory Martinez,23.0,34.659849
3,seminar,Pension,Kathryn Maxwell,23.0,71.253490
4,seminar,Mortgage,Leah Nelson,23.0,57.169339
5,seminar,Pension,Angela Sanchez,23.0,75.004685
6,seminar,Mortgage,Michelle Roy,23.0,68.208093
7,gift,Savings,David Lewis,20.0,22.490020
8,seminar,Pension,Brandy Murphy,23.0,86.160881
9,seminar,Mortgage,Alison Chen,23.0,38.122278


In [12]:
report_bd.groupby(['channel', 'product']).count()

customer  cost  revenue
channel product                          
gift    Savings          2     2        2
seminar Mortgage        34    34       34
        Pension         73    73       73
        Savings         20    20       20

Using the sample, this has given us the rough outline of the optimization. Using these figures, replicate using non-linear minimization.

In [ ]:
n_obs_orig = n_obs_new
n_obs_new = 10000

In [ ]:
product_probs_orig = pandas.read_csv('offers_ibm_pivot.csv')
n_obs_original = product_probs_orig.shape[0]

product_probs = pandas.read_csv('sample_data_10000.csv')
# product_probs = product_probs[product_probs.index > product_probs.shape[0] - n_obs_new]
product_probs = product_probs[product_probs.index < n_obs_new]
n_obs = product_probs.shape[0]

adjustment_factor = n_obs/n_obs_original
availableBudget = availableBudget*adjustment_factor

product_probs.rename(columns={'Unnamed: 0': 'customerid'}, inplace=True)
product_probs.head()

In [ ]:
num_customers = product_probs.shape[0]

In [ ]:
import scipy.optimize as optimize

In [ ]:
offer_scale = int(n_obs_new/n_obs_orig)

# get the offers from the original optimization by product and channel
sample_counts = pandas.pivot_table(report_bd, index='channel', columns='product', values='customer', 
                                   aggfunc=len, fill_value=0)

offers = sample_counts.stack()*offer_scale

# offers = []

# for i, ival in enumerate(channels):
#    for j, jval in enumerate(products):
#        offers.append( sample_counts.iloc[i, j])

### Note: need to update with offer_scale
         

In [ ]:
offers

Calculate the profit vectors

In [ ]:
product_profit = product_probs[products]*productValue
product_profit_0 = product_profit*factor[0]
product_profit_1 = product_profit*factor[1]
product_profit_2 = product_profit*factor[2]
product_profit_0.columns = [pp + ' ' + channels[0] for pp in products]
product_profit_1.columns = [pp + ' ' + channels[1] for pp in products]
product_profit_2.columns = [pp + ' ' + channels[2] for pp in products]
product_profit = pandas.concat([product_profit_0, product_profit_1, product_profit_2], axis=1)

Write data to CSV for separate analysis.

In [ ]:
product_profit.to_csv('product_profit.csv')

### Dual Minimization using Nonlinear Minimization Function

In R I used the function `nlm` which is non-linear minimization. It takes a function to minimize and starting parameter values for the minimization as arguments.

```{r}
u_init <- offers*0
out <- nlm(dual, u_init, print.level = 1)
```

### Keep dual minimum and estimates.
```{r}
mindual <- out$minimum
u <- out$estimate
mindual
u
```

In Python I will use `scipy.optimize.minimize`.

In [ ]:
def dual(u):
    '''
    Dual function to optimize
    '''
    d = product_profit.sub(u)
    v = d.max(axis=1)
    v[v<0] = 0
    y = np.dot(offers, u) + v.sum()
    return(y)

In [ ]:
## Test dual function

u_test = [11.2, 15, 6.02, 19.5, 0, 4.98, 2.23, 7.75, 50.2, 23.2, 9.09, 35.2]
dual(u_test)

In [ ]:
u_test = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dual(u_test)

Test the following: 

  1. `.sub(u)` for non-zero `u`. **Tick.**
  2. `d.max(axis=1).` **Tick.**
  3. `np.dot(offers, u)`.

In [ ]:
product_profit.max(axis=1).head(10)
    

Tried the following optimization methods:

  * **default:** not successful -- 'Desired error not necessarily achieved due to precision loss.'
  * **Nelder-Mead:** successful

In [ ]:
u_init = offers*0


In [ ]:
# test the function which we are optimizing
# product_profit.sub(u_init.tolist())
# dual(u_init.tolist())


# result = optimize.minimize(dual, u_init, method="Nelder-Mead")
# result = optimize.minimize(dual, u_init, method="Powell")
# result = optimize.minimize(dual, u_init) # no success
result = optimize.minimize(dual, u_init, method="CG")

In [ ]:
result.success

In [ ]:
result.x

In [ ]:
result

In [ ]:
u = result.x
d = product_profit.sub(u) 
v = d.max(axis = 1)
v[v<0] = 0
ndx = np.argsort(-v)

In [ ]:
d['customerid'] = product_probs['customerid']

In [ ]:
# ```{r}
# d_DT <- data.table(d)
# d_DT[, customerid := product_probs$customerid]
# d_DT[, v := v]

# d_DT_melt <- melt(d_DT, id.vars = c("customerid", "v"))
# d_DT_alloc <- d_DT_melt[order(customerid, -value)][, lapply(.SD, head, 1), by = .(customerid)][seq(sum(offers))]

# # check counts
# d_DT_alloc[, .N, by = .(variable)]
# ```

In [ ]:
d['customerid'] = product_probs['customerid']
d['v'] = v

# melt it ... is this stack?